In [1]:
import treeflow.libsbn
import treeflow.model

newick_file = 'demo-data/dengue.newick'
fasta_file = 'demo-data/dengue.fasta'

inst = treeflow.libsbn.get_instance(newick_file)
q_tree = treeflow.model.construct_tree_approximation(newick_file, inst=inst)

In [2]:
q_tree.sample()

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [3]:
bij = q_tree.model['heights'].distributions[1].bijector.bijectors[0]
base = q_tree.model['heights'].distributions[1].distribution

In [4]:
bij._forward_1d_numpy(base.sample().numpy())

TypeError: set_node_heights_via_height_ratios(): incompatible function arguments. The following argument types are supported:
    1. (self: libsbn.RootedTree, arg0: libsbn.vector_double) -> None

Invoked with: <libsbn.RootedTree object at 0x7f17738e1db0>, array([-1.1994077e+00,  3.7405714e-01,  2.1013618e-04,  1.2124320e+00,
        4.4476125e-01,  4.6085200e+00,  1.3260307e+00, -1.7412281e+00,
        3.4171925e+00,  2.8585455e+00,  1.4044667e+00,  1.2141166e+00,
       -1.3889534e+00,  1.4991452e+00, -1.9379904e+00,  6.9900837e+00],
      dtype=float32)